# Setup

In [2]:
# additional google colab setup
import sys


def colab_install():
    import torch
    if not torch.cuda.is_available():
      print("CUDA is not available. \nPick a GPU before running this notebook. \nGo to 'Runtime' -> 'Change runtime type' to do this.")
      return 
    %pip install transformers
    %pip install datasets
    %pip install peft
    %pip install bitsandbytes
    return


if "google.colab" in sys.modules:
    print("Running in Google Colab")
    # Install required packages
    colab_install()
else:
    print("Not running in Google Colab")

Not running in Google Colab


In [3]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, TaskType, prepare_model_for_kbit_training, get_peft_model
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

transformers.set_seed(24)

In [4]:
# TODO remove
DEBUG = True
if DEBUG:
    %cd survai-finetuning/

/home/jovyan/work/survai-finetuning


/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Data preperation

In [5]:
# download dataset
!curl -L -o 2016_anes_argyle.pkl https://github.com/tobihol/survai-finetuning/raw/main/2016_anes_argyle.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  149k  100  149k    0     0   439k      0 --:--:-- --:--:-- --:--:--  439k


In [6]:
df_survey = pd.read_pickle("2016_anes_argyle.pkl")
df_survey

,race,discuss_politics,ideology,party,church_goer,age,gender,political_interest,patriotism,state,ground_truth
0,white,Yes,NaN,a strong Republican,attend church,29,man,somewhat,extremely good,Louisiana,Trump
1,white,Yes,slightly conservative,a weak Republican,do not attend church,26,man,very,extremely good,Arkansas,Trump
2,white,No,NaN,an independent who leans Democratic,do not attend church,23,man,not very,moderately good,Mississippi,Non-voter
3,white,Yes,NaN,an independent who leans Republican,attend church,58,man,somewhat,extremely good,Tennessee,Trump
4,white,No,moderate,an independent who leans Democratic,attend church,38,woman,not very,extremely good,Ohio,Non-voter
...,...,...,...,...,...,...,...,...,...,...,...
4265,white,Yes,NaN,a strong democrat,attend church,37,woman,not very,extremely good,Virginia,Clinton
4266,white,Yes,moderate,a strong democrat,attend church,82,man,somewhat,extremely good,Virginia,Trump
4267,white,No,moderate,a weak Democrat,attend church,27,man,not very,a little good,Georgia,Non-voter
4268,black,Yes,conservative,a strong democrat,attend church,39,woman,somewhat,neither good nor bad,North Carolina,Clinton


In [7]:
# descriptive statistics
df_survey.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4270 entries, 0 to 4269
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   race                4060 non-null   object
 1   discuss_politics    3645 non-null   object
 2   ideology            3303 non-null   object
 3   party               4247 non-null   object
 4   church_goer         4251 non-null   object
 5   age                 4149 non-null   UInt8 
 6   gender              4218 non-null   object
 7   political_interest  3638 non-null   object
 8   patriotism          3645 non-null   object
 9   state               4242 non-null   object
 10  ground_truth        4270 non-null   object
dtypes: UInt8(1), object(10)
memory usage: 375.3+ KB


In [8]:
features = [
    "race",
    "discuss_politics",
    "ideology",
    "party",
    "church_goer",
    "age",
    "gender",
    "political_interest",
    "patriotism",
    "state",
]
label = "ground_truth"

In [9]:
# we tread missing values as a category 
df_survey_processed = (
    df_survey
    .astype({"age": str})
    .fillna("missing")
)
df_survey_processed

,race,discuss_politics,ideology,party,church_goer,age,gender,political_interest,patriotism,state,ground_truth
0,white,Yes,missing,a strong Republican,attend church,29,man,somewhat,extremely good,Louisiana,Trump
1,white,Yes,slightly conservative,a weak Republican,do not attend church,26,man,very,extremely good,Arkansas,Trump
2,white,No,missing,an independent who leans Democratic,do not attend church,23,man,not very,moderately good,Mississippi,Non-voter
3,white,Yes,missing,an independent who leans Republican,attend church,58,man,somewhat,extremely good,Tennessee,Trump
4,white,No,moderate,an independent who leans Democratic,attend church,38,woman,not very,extremely good,Ohio,Non-voter
...,...,...,...,...,...,...,...,...,...,...,...
4265,white,Yes,missing,a strong democrat,attend church,37,woman,not very,extremely good,Virginia,Clinton
4266,white,Yes,moderate,a strong democrat,attend church,82,man,somewhat,extremely good,Virginia,Trump
4267,white,No,moderate,a weak Democrat,attend church,27,man,not very,a little good,Georgia,Non-voter
4268,black,Yes,conservative,a strong democrat,attend church,39,woman,somewhat,neither good nor bad,North Carolina,Clinton


## Prompt Design

In [10]:
instruction = (
    "Please perform a classification task. "
    + "Given the 2016 survey answers from the American National Election Studies, "
    + "return which candiate the person voted for. "
    + "Return a label from ['Trump', 'Clinton', 'Non-voter'] only without any other text.\n"
)
print(instruction)

Please perform a classification task. Given the 2016 survey answers from the American National Election Studies, return which candiate the person voted for. Return a label from ['Trump', 'Clinton', 'Non-voter'] only without any other text.



In [11]:
column_name_map = {
    "race": "Race",
    "discuss_politics": "Discusses politics",
    "ideology": "Ideology",
    "party": "Party",
    "church_goer": "Church",
    "age": "Age",
    "gender": "Gender",
    "political_interest": "Political interest",
    "patriotism": "American Flag",
    "state": "State",
    "ground_truth": "Vote",
}

def create_prompt(row):
    prompt = instruction
    prompt += "\n".join([f"{column_name_map[k]}: {v}" for k, v in row.items()])
    return prompt

text_series = df_survey_processed[features].apply(create_prompt, axis="columns")
label_series = df_survey_processed[label]

df_prompts = pd.DataFrame({"text": text_series, "label": label_series})
df_prompts

,text,label
0,Please perform a classification task. Given th...,Trump
1,Please perform a classification task. Given th...,Trump
2,Please perform a classification task. Given th...,Non-voter
3,Please perform a classification task. Given th...,Trump
4,Please perform a classification task. Given th...,Non-voter
...,...,...
4265,Please perform a classification task. Given th...,Clinton
4266,Please perform a classification task. Given th...,Trump
4267,Please perform a classification task. Given th...,Non-voter
4268,Please perform a classification task. Given th...,Clinton


In [12]:
print(df_prompts.text.iloc[0])

Please perform a classification task. Given the 2016 survey answers from the American National Election Studies, return which candiate the person voted for. Return a label from ['Trump', 'Clinton', 'Non-voter'] only without any other text.
Race: white
Discusses politics: Yes
Ideology: missing
Party: a strong Republican
Church: attend church
Age: 29
Gender: man
Political interest: somewhat
American Flag: extremely good
State: Louisiana


In [13]:
print(df_prompts.label.iloc[0])

Trump


## Train/Test Split

In [14]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_survey_processed, test_size=0.2, random_state=24)
dataset_cls = DatasetDict({
    "train": Dataset.from_pandas(df_train, preserve_index=False),
    "test": Dataset.from_pandas(df_test, preserve_index=False),
})
dataset_cls

DatasetDict({
    train: Dataset({
        features: ['race', 'discuss_politics', 'ideology', 'party', 'church_goer', 'age', 'gender', 'political_interest', 'patriotism', 'state', 'ground_truth'],
        num_rows: 3416
    })
    test: Dataset({
        features: ['race', 'discuss_politics', 'ideology', 'party', 'church_goer', 'age', 'gender', 'political_interest', 'patriotism', 'state', 'ground_truth'],
        num_rows: 854
    })
})

In [15]:
df_train, df_test = train_test_split(df_prompts, test_size=0.2, random_state=24)
dataset_llm = DatasetDict({
    "train": Dataset.from_pandas(df_train, preserve_index=False),
    "test": Dataset.from_pandas(df_test, preserve_index=False),
})
dataset_llm

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3416
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 854
    })
})

# Loading the model

Model selection:
- https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard
- https://lmarena.ai/
- https://crfm.stanford.edu/helm/
    - Imputation Benchmark: https://crfm.stanford.edu/helm/classic/latest/#/groups/entity_data_imputation

Reproducibility:
- (TODO how model revisions)
- LLM training/inference in the wild west: the are a ton of differnt libraries/wrappers where each one can implement different changes your evaluations results. (TODO add tokenisation problem that I had during my thesis)

In [16]:
# model_id = "EleutherAI/pythia-70m"
model_id = "unsloth/Llama-3.2-1B-Instruct"
# TODO add revision number

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side="left",
    trust_remote_code=True,
)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


# dataset = dataset.map(
#     lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True
# )

# dataset = dataset.remove_columns(["text", "label"])
# dataset

In [17]:
def instruct_tokenize_function(examples):
    prompt = [
        {"role": "user", "content": examples["text"]},
    ]
    prompt.append(
        {
            "role": "assistant",
            "content": examples["label"],
        }
    )
    inputs_ids = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=False,
    )
    attention_mask = np.ones_like(inputs_ids)
    return {
        "input_ids": inputs_ids,
        "attention_mask": attention_mask,
    }


def basic_tokenize_function(examples):
    prompt = f"{examples['text']} \nVote: {examples['label']} {tokenizer.eos_token}"
    return tokenizer(prompt)


tokenized_dataset_llm = dataset_llm.map(basic_tokenize_function).remove_columns(
    ["text", "label"]
)
tokenized_dataset_llm

Map:   0%|          | 0/3416 [00:00<?, ? examples/s]

Map:   0%|          | 0/854 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3416
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 854
    })
})

In [18]:
# load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)

model = prepare_model_for_kbit_training(model)

if getattr(model.config, "pad_token_id") is None:
    model.config.pad_token_id = tokenizer.pad_token_id

In [19]:
lora_rank = 8
lora_alpha = 8

lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules="all-linear",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.config.use_cache = False

trainable params: 5,636,096 || all params: 1,241,450,496 || trainable%: 0.4540


In [ ]:
from typing import Tuple


def instruct_tokenization(
    data: DatasetDict,
    tokenizer: AutoTokenizer,
) -> Tuple[DatasetDict, Dataset]:
    def tokenize_function(examples, is_inference=False):
        prompt = [
            {"role": "user", "content": examples["text"]},
        ]
        if not is_inference:
            prompt.append(
                {
                    "role": "assistant",
                    "content": examples["label"],
                }
            )
        inputs_ids = tokenizer.apply_chat_template(
            prompt,
            add_generation_prompt=is_inference,
        )
        attention_mask = np.ones_like(inputs_ids)
        return {
            "input_ids": inputs_ids,
            "attention_mask": attention_mask,
        }

    column_names = list(data.column_names.values())[0]
    training_data = data.map(tokenize_function, remove_columns=column_names)
    from functools import partial

    inference_data = data.map(
        partial(tokenize_function, is_inference=True), remove_columns=column_names
    )

    answer_tokens = list(
        {
            training_ids[len(inference_ids)]
            for inference_ids, training_ids in zip(
                inference_data["train"]["input_ids"]
                + inference_data["test"]["input_ids"],
                training_data["train"]["input_ids"]
                + training_data["test"]["input_ids"],
            )
        }
    )
    assert len(answer_tokens) == len(
        set(data["test"]["label"] + data["train"]["label"])
    )

    return training_data, inference_data, answer_tokens


training_data, inference_data, answer_tokens = instruct_tokenization(
    dataset_llm, tokenizer
)

In [49]:
import evaluate
from sklearn import metrics
from functools import partial

# TODO: make other metrics work
hf_metrics = [
    evaluate.load("accuracy"),
    # evaluate.load("f1"),
    # evaluate.load("precision"),
    # evaluate.load("recall"),
    # evaluate.load("confusion_matrix"),
]
sklearn_metrics = {
    # "accuracy": metrics.accuracy_score,
    # "balanced_accuracy": metrics.balanced_accuracy_score,
    # "f1_weighted": partial(metrics.f1_score, average="weighted"),
    # "confusion_matrix": metrics.confusion_matrix,
}

pred_slice_ids = [
    (len(inference_ids), len(training_ids) - 1)
    for inference_ids, training_ids in zip(
        inference_data["test"]["input_ids"], training_data["test"]["input_ids"]
    )
]  # NOTE: the -1 accounts for the eos token, which is not present for the generation data


def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    logits = logits[:, :, answer_tokens].argmax(dim=-1)

    return torch.tensor(
        answer_tokens,
        device="cuda",
    )[logits]


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics but we need to shift the labels
    preds = np.pad(preds, ((0, 0), (1, 0)), mode="constant", constant_values=-100)
    labels = np.pad(labels, ((0, 0), (0, 1)), mode="constant", constant_values=-100)

    def get_slice(y):
        return [
            [token for token, label_token in zip(row, label) if label_token != -100][
                start_id:end_id
            ]
            for (start_id, end_id), row, label in zip(pred_slice_ids, y, labels)
        ]

    y_true = get_slice(labels)
    y_pred = get_slice(preds)
    # accuracy based on the first token of the vote
    y_true = [row[0] for row in y_true]
    y_pred = [row[0] for row in y_pred]

    results = {}
    for metric in hf_metrics:
        results |= metric.compute(predictions=y_pred, references=y_true)
    for metric_name, metric_func in sklearn_metrics.items():
        results[metric_name] = metric_func(y_true=y_true, y_pred=y_pred)
    return results

In [51]:
import wandb
from datetime import datetime

now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
run_name = f'{model_id}_{now}'

# wandb.init(
#     mode='disabled',
# )

wandb.init(
    project="survai-finetuning",
    name=run_name,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=training_data["train"].select(range(80)),
    eval_dataset=training_data["test"].select(range(20)),
    args=transformers.TrainingArguments(
        output_dir="./results",
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        fp16=True,
        optim="paged_adamw_8bit",

        # train/eval settings
        num_train_epochs=3,
        do_eval=True,
        evaluation_strategy="epoch",

        # logging
        report_to="wandb",
        run_name="test-run",
    ),
    # tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
)

trainer.evaluate()
trainer.train()

wandb.finish()

/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,No log,0.262326,0.003600,0.850000
2,No log,0.253618,0.003600,0.800000
3,No log,0.252551,0.003600,0.750000


/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/home/jovyan/work/survai-finetuning/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


eval/accuracy,▁█▆▃
eval/loss,▆█▂▁
eval/model_preparation_time,▁▁▁▁
eval/runtime,█▂▁▆
eval/samples_per_second,▁▇█▃
eval/steps_per_second,▁▇█▃
train/epoch,▁▅██
train/global_step,▁▃▆██
eval/accuracy,0.75
eval/loss,0.25255
eval/model_preparation_time,0.0036


## Answer Extraction

<!-- - https://arxiv.org/pdf/2307.09702, https://github.com/dottxt-ai/outlines -->
Different modles for answer extraction:
- https://blog.eleuther.ai/multiple-choice-normalization/
- https://github.com/huggingface/lighteval

Problems:
- One token solutions:
    - less compute intensive
    - do not require normalisation
    - only works if all first token are destinct
- Multi token solutions:
    - more compute intensive (muliplies by number of lables)
    - some requires normalisation


# Things we did not do

- hyperparameter search
- cross validation
- running multiple seeds 